Link: [geeksforgeeks](https://www.geeksforgeeks.org/machine-learning/multi-armed-bandit-problem-in-reinforcement-learning/)

[github](https://gibberblot.github.io/rl-notes/single-agent/multi-armed-bandits.html)

### Bài toán 
Động lực: không thể chọn explore và exploit cùng lúc 

Giải pháp: lựa chọn Hành động theo giới hạn trên khoảng tin cậy (Upper Confidence Bound - UCB)
$$ Confident\_Interval = \argmax_a \Big(Q_t(a) + c \sqrt{\frac{\ln(t)}{N_t(a)}} \Big) $$
Khoảng tin cậy cho ta biết độ tin cậy giá trị hành động thực tế của hành động A nằm ở đâu trong vùng này
- Nếu vùng này nhỏ: chắc chắc giá trị của A gần với ước tính 
- Ngược lại, không hcawcs chắn giá trị hành động A gần với ước tính 

Nguyên tắc: lạc quan với sự không chắc chắn, tức là nếu không biết chọn hành động nào thì chọn hành động có Upper Bound lớn nhất. 

### Cơ sở toán học: 
(Gemini)

- Bất đẳng thức Hoeffding: nôm na là cho thấy xác suất để giá trị thực $\mu$ vượt quá giá trị trung bình quan sát $\bar{X}$ một khoảng $\Delta$ sẽ giảm rất nhanh theo hàm mũ khi mẫu $n$ tăng lên: 
$$S_n = \sum_{i=1}^{n} X_i$$
$$ P(S_n - E[S_n] \geq t) \leq \exp \left( -\frac{2t^2}{\sum (b_i - a_i)^2} \right) (1) $$
$$ P(\mu - \bar{X} > \Delta) \le \exp(-2n\Delta^2) (2)$$
<br> $\bar{X} \to \frac{S_n}{n}$ : trung bình mẫu của $n$ bnn độc lập
<br> $\mu \to E[\bar{X}]/n$ : giá trị kỳ vọng thực 
<br> $\Delta \to t/n$ : Độ lệch 

Mặc định $X_i \in [0,1]$ nên $(b_i-a_i)^2 = 1$ nên $\sum_{i=1}^{n} (b_i-a_i)^2 = n$

Thay hết mấy cái trên vào (1), đồng thời xoay chiều để tìm "trần", ta suy ra 2

Tìm upper bound: 
$$ P(\mu > \bar{X} + \Delta) \le \exp(-2n\Delta^2) = p$$
Lấy loganepe 2 vế: 
$$ -2n \Delta^2 = ln(p) \implies \Delta = \sqrt{\frac{\ln(1/p)}{2n}} $$
Làm sao để xác suất rủi ro giảm dần theo $t$ ? Chọn $p = \frac{1}{t^{\alpha}}$. Vậy 
$$\Delta = \sqrt{\frac{\alpha \ln(t)}{2n}} = c \sqrt{\frac{\ln(t)}{N_t(a)}}$$

- Khoảng tin cậy: $Upper\_Bound = \bar{X_a} + U_a$

Giá trị được chọn phổ biến là: 
$$A_t = \arg \max_{a} \left[ Q_t(a) + c \sqrt{\frac{\ln t}{N_t(a)}} \right]$$

- Sự hội tụ và luật số lớn: Ý tưởng là khi kéo một tay máy càng nhiều lần, độ tin cậy sẽ nhỏ lại 
- Lý thuyết Regret (Hối tiếc): Mức độ "sai lầm" của thuật toán tuân theo hàm log, tăng chậm theo thời gian, giúp tối ưu về mặt toán học (ít nhất là hơn epsilon-greedy)

### Triển khai 

In [1]:
import numpy as np

In [2]:
class UCB:
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
        self.total_counts = 0
    
    def select_arm(self):
        ucb_values = self.values + np.sqrt(2 * np.log(self.total_counts+1) / (self.counts + 1e-5))
        return np.argmax(ucb_values)
    
    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] += 1
        self.total_counts += 1
        n = self.counts[chosen_arm]
        value_past = self.values[chosen_arm]
        self.values[chosen_arm] = (n-1) / n * value_past + 1/n * reward

In [12]:
''' Mẫu chạy thử '''
n_arms = 10
T = 1000

''' 
Thiết lập trạng thái ban đầu 
true_means : kỳ vọng reward của một máy 
'''
true_means = np.random.randn(n_arms)
print(f"Actual mean rewards for each arm: \n {np.round(true_means, 2)}")
print(f"The best arm is #{np.argmax(true_means)} with a mean of {np.max(true_means):.2f} \n")
agent = UCB(n_arms)
total_reward = 0

Actual mean rewards for each arm: 
 [-0.47 -0.94  0.8   2.03  0.45 -0.07 -0.35  1.59  0.51 -1.23]
The best arm is #3 with a mean of 2.03 



In [14]:
for t in range(T):
    arm_to_pull = agent.select_arm()
    reward = np.random.randn() + true_means[arm_to_pull]
    agent.update(arm_to_pull, reward)
    total_reward += reward

print(f"Values: \n {np.round(agent.values, 2)} \n")
print(f"Total Reward: {total_reward:.2f}")

vars

Values: 
 [-1.58 -1.95  0.9   2.04 -0.01 -1.01 -1.17  0.89  0.78 -1.67] 

Total Reward: 4006.51


<function vars>